In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import scipy.io
import cv2
import glob
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import torch.utils.data
%matplotlib inline



/home/dev/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:

df=pd.read_csv('/home/dev/Downloads/mpii_dataset.csv')



In [3]:
from sklearn.model_selection import train_test_split
df.drop(['Category','Activity'],1)
df = df.sample(n = len(df)).reset_index(drop=True)

/tmp/ipykernel_3545/934581437.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['Category','Activity'],1)


In [4]:
df.head()
from torch.utils.data import Dataset


In [5]:
transferdata_df = df.iloc[- 1400:, 0:]

test_df = pd.DataFrame()
test_df= test_df.append(transferdata_df)
train_df =df.drop(transferdata_df.index)
test_df = test_df.reset_index(drop=True)
test_df.head()

/tmp/ipykernel_3545/1936806666.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df= test_df.append(transferdata_df)


,Unnamed: 0,NAME,r ankle_X,r ankle_Y,r knee_X,r knee_Y,r hip_X,r hip_Y,l hip_X,l hip_Y,...,r shoulder_Y,l shoulder_X,l shoulder_Y,l elbow_X,l elbow_Y,l wrist_X,l wrist_Y,Scale,Activity,Category
0,0,089586694.jpg,415,488,491,445,561,355,595,367,...,234,640,266,633,327,645,381,2.203430,shuffleboard,sports
1,0,008564467.jpg,264,1075,184,800,398,622,524,629,...,383,649,400,765,559,770,549,5.362376,reclining,inactivity quiet/light
2,0,031491844.jpg,411,406,401,348,396,304,424,301,...,241,390,219,449,243,442,278,1.709231,"bicycling, mountain",bicycling
3,0,063141892.jpg,330,311,236,288,264,366,301,346,...,253,291,245,317,271,326,280,1.989106,"yoga, Power",conditioning exercise
4,0,038504124.jpg,724,802,799,723,762,619,801,629,...,472,883,504,926,554,969,578,3.140976,"skiing, downhill",winter activities


In [32]:
img_loc=("/home/dev/Downloads/Images/images/")
#imgArray = np.array([])
#files = glob.glob(img_loc+'*.jpg')
imgcomb=[]


images=[]       
trainimglabels=train_df.loc[:,'r ankle_X':'Scale']
trainfilenames=train_df['NAME']

testimglabels=test_df.loc[:,'r ankle_X':'Scale']
testfilenames=test_df['NAME']

class Imagedataset(Dataset):
    def __init__(self,fnames,labels,transform=None):
        self.fnames=fnames
        self.labels=labels
        self.transform=transform
    
    def __len__(self):
        return len(self.fnames)
    
    def NormaliseData(self,img,idx):
        #print(self.labels.head())
        height, width = img.shape
        #print(self.labels.head())
        for i in range(0,31,2):
        
            self.labels.iloc[idx,i]=(self.labels.iloc[idx,i]*(256/width))
            modscale=self.labels.iloc[idx,-1]*(400/200)
            self.labels.iloc[idx,i+1]=(self.labels.iloc[idx,i+1]*(256/height))
            
        #print(self.labels.head())
        normImg=cv2.resize(img,(256,256))
        return normImg


    def __getitem__ (self, idx):
        Rimg=cv2.imread(img_loc+self.fnames[idx])
        Gimg=cv2.cvtColor(Rimg,cv2.COLOR_BGR2GRAY)
        Gimg=self.NormaliseData(Gimg,idx)
        #for i in range(0,31,2):
         #   y=self.labels.iloc[idx,i+1]
          #  x=self.labels.iloc[idx,i]
           # cv2.circle(Gimg,(int(x),int(y)),5,(0,255,0),-1)
        #cv2.imshow("gay", Gimg)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        Gimg = Gimg.astype(np.float32) / 255.
        if self.transform:
            Gimg = self.transform(Gimg)
       
        helper=self.labels.iloc[idx, :-1]
       
        target=helper.values
        target=target.reshape(-1,2)
     
        return (Gimg,target)

    
train_dataset= Imagedataset(trainfilenames,trainimglabels)
test_dataset= Imagedataset(testfilenames,testimglabels)

train_loader=DataLoader(train_dataset,batch_size=15, shuffle=True, num_workers=4)
test_loader=DataLoader(test_dataset,batch_size=15, shuffle=True)




In [7]:
data, target= train_dataset.__getitem__(1)
asdf=pd.DataFrame(data)
print(data.shape)


(240, 426)


In [8]:
train_dataset.fnames.iloc[4]
import itertools

In [44]:
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1)
        self.avg_pool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.fc1 = nn.Linear(64*64*64,64)
        self.fc2 = nn.Linear(64,16*2)

    def forward(self, X):
        X = self.conv1(X)
        X = F.relu(X)
        X = F.max_pool2d(X,kernel_size=2,stride=2)
        X = self.conv2(X)
        X = F.relu(X)
     
        X = F.max_pool2d(X,kernel_size=2,stride=2)
        X = torch.flatten(X,1)
        X = self.fc1(X)
        X = F.relu(X)
        X = self.fc2(X)
        X=X.view(-1,16,2)
        output = F.log_softmax(X, dim=1)
        return output
    
torch.manual_seed(101)
model = ConvolutionalNetwork().to(dev)
print(dev)

cuda:0


In [38]:

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [40]:
def train(model, dev, train_loader, optimizer, epoch):
  model.train()
  
  for batch_idx in range(0,len(train_loader),15):
    print("dik")
    #start_idx = batch_idx
    #end_idx = min(len(train_loader), batch_idx + 20)
    batch=list(itertools.islice(train_loader,15))
    for data, target in batch:
      print("coc")
      data=data.unsqueeze(1)
      print(data.shape ," MIAGECI ",target.shape)
      data, target = data.to(dev), target.to(dev)
      optimizer.zero_grad()
      output = model(data)
      loss = criterion(output, target)
      loss.backward()
      optimizer.step()
    
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.3f}'.format(
      epoch, batch_idx * len(data), len(train_loader.dataset),
      100. * batch_idx / len(train_loader), loss.item()))
    
        

In [31]:

train_dataset.__getitem__(0)


(array([[0.14901961, 0.14901961, 0.47058824, ..., 0.23529412, 0.5647059 ,
         0.5568628 ],
        [0.1254902 , 0.10980392, 0.42352942, ..., 0.25490198, 0.60784316,
         0.5882353 ],
        [0.10588235, 0.08627451, 0.3764706 , ..., 0.2901961 , 0.627451  ,
         0.61960787],
        ...,
        [0.3647059 , 0.36862746, 0.38039216, ..., 0.36862746, 0.3764706 ,
         0.3647059 ],
        [0.38039216, 0.38039216, 0.39215687, ..., 0.36862746, 0.36862746,
         0.36862746],
        [0.36078432, 0.36862746, 0.3764706 , ..., 0.35686275, 0.35686275,
         0.3529412 ]], dtype=float32),
 array([[169.8       , 253.15555556],
        [152.        , 192.35555556],
        [157.8       , 140.08888889],
        [142.        , 140.8       ],
        [135.4       , 187.02222222],
        [140.8       , 252.44444444],
        [150.        , 140.44444444],
        [149.2       ,  68.26666667],
        [148.76848   ,  54.77752889],
        [147.23152   ,   6.73358222],
        [158.2

In [13]:
torch.cuda.empty_cache()

In [45]:
for epoch in range(1, 6):
    train(model, dev, train_loader, optimizer, epoch)

dik
coc
torch.Size([15, 1, 256, 256])  MIAGECI  torch.Size([15, 16, 2])
coc
torch.Size([15, 1, 256, 256])  MIAGECI  torch.Size([15, 16, 2])
coc
torch.Size([15, 1, 256, 256])  MIAGECI  torch.Size([15, 16, 2])
coc
torch.Size([15, 1, 256, 256])  MIAGECI  torch.Size([15, 16, 2])
coc
torch.Size([15, 1, 256, 256])  MIAGECI  torch.Size([15, 16, 2])
coc
torch.Size([15, 1, 256, 256])  MIAGECI  torch.Size([15, 16, 2])
coc
torch.Size([15, 1, 256, 256])  MIAGECI  torch.Size([15, 16, 2])
coc
torch.Size([15, 1, 256, 256])  MIAGECI  torch.Size([15, 16, 2])
coc
torch.Size([15, 1, 256, 256])  MIAGECI  torch.Size([15, 16, 2])
coc
torch.Size([15, 1, 256, 256])  MIAGECI  torch.Size([15, 16, 2])
coc
torch.Size([15, 1, 256, 256])  MIAGECI  torch.Size([15, 16, 2])
coc
torch.Size([15, 1, 256, 256])  MIAGECI  torch.Size([15, 16, 2])
coc
torch.Size([15, 1, 256, 256])  MIAGECI  torch.Size([15, 16, 2])
coc
torch.Size([15, 1, 256, 256])  MIAGECI  torch.Size([15, 16, 2])
coc
torch.Size([15, 1, 256, 256])  MIAGECI  

KeyboardInterrupt: 